# Library District Revenue
This analysis estimates the potential revenue that Boulder's proposed library district is expected to produce on a per-parcel basis. All data is open and can be found at the [Boulder County Assessor](https://www.bouldercounty.org/property-and-land/assessor/data-download/), [Boulder County Open Data Portal](https://opendata-bouldercounty.hub.arcgis.com/), and [City of Boulder Open Data Portal](https://open-data.bouldercolorado.gov/). We use open data in this analysis to encourage transparency and enable reproducibility.

In [ ]:
from functools import reduce

import pandas as pd
import numpy as np

## Gather Data URLs

In [ ]:
# Assessor's tax information
ACCOUNTS_URL = "https://assessor.boco.solutions/ASR_PublicDataFiles/Account_Parcels.csv"
OWNERS_URL = "https://assessor.boco.solutions/ASR_PublicDataFiles/Owner_Address.csv"
VALUES_URL = "https://assessor.boco.solutions/ASR_PublicDataFiles/Values.csv"

# Library district boundary
DISTRICT_URL = "https://maps.bouldercolorado.gov/arcgis/rest/services/general/LibraryDistrict/MapServer/0"

# Boulder county data
PARCELS_URL = "https://maps.bouldercounty.org/arcgis/rest/services/Emap/BOCO_Parcels/MapServer/0"
MUNI_URL = "https://maps.bouldercounty.org/arcgis/rest/services/PLANNING/LUC_ZoningDistricts_DQMunicipalities/MapServer/0"
FIRE_URL = "https://maps.bouldercounty.org/arcgis/rest/services/HAZARD/FIRE_WILDFIRE_HISTORY/MapServer/0"

## Process Tax Data

In [ ]:
# Simple function to handle csv's from the county
def process_csv(src:str, col_map:dict, col_types:dict=None) -> pd.DataFrame:
    """Process csv data.

    Maps old column names to new column names, and optionally specifies
    column types to avoid mixed types. It will also drop duplicates once
    columns have been remapped.

    Parameters
    ----------
    src : str
        csv source, URL or file
    col_map : dict
        column mapping, where dict keys are old names and values are the
        new names.
    col_types : dict, optional
        column types, by default None

    Returns
    -------
    pd.DataFrame
        A pandas dataframe of the csv contents
    """
    df = pd.read_csv(src, usecols=list(col_map.keys()), dtype=col_types)
    df.rename(col_map, axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    return df


### Accounts and Parcel Numbers
According to the Boulder County Assessor's [data dictionary](https://assets.bouldercounty.org/wp-content/uploads/2017/02/ar-property-data-download-help.pdf), "The only tables that contain only one row for every active real
property account in our database are the Account_Parcels table and the Values table". This means that, in both the Account_parcels and Values tables, `strap` is never duplicated, and therefore these two tables can be joined 1:1. The Account_Parcels table is dead simple, so no cleaning has to be done.

In [ ]:
# Import values
col_names = {"strap": "STRAP", "Parcelno": "PARCELNUM"}
col_types = {"strap": str, "Parcelno": str}
acct_df = process_csv(ACCOUNTS_URL, col_names, col_types)


### Values
Assessed values are calculated by multiplying the total actual value by the tax rate for the property:

```
(Assessed Value) = (Actual Value) * (Tax Rate)
```

See this [detailed webpage](https://www.bouldercounty.org/property-and-land/assessor/tax-calculation/) for current and prior tax rates. We rely on the county's calculations for estimating revenue.

In [ ]:
# Import values
col_names = {"strap": "STRAP", "tax_yr": "TAXYR",
             "totalActualVal": "ACTUALVAL", "totalAssessedVal": "ASSESSVAL"}
col_types = {"strap": str, "tax_yr": str,
             "totalActualVal": np.int32, "totalAssessedVal": np.int32}
val_df = process_csv(VALUES_URL, col_names, col_types)


### Owners and Addresses
We would not normally need this table for our analysis, but the county provides mill levy rates and account types here. While each account has one mill levy, it is not the case that every account has only one owner. In other words, one `strap` ID can show up multiple times in this table, either because there are multiple owners for one account (split ownership) or there are multiple situs addresses. For simplicity's sake, if `strap` comes up twice, we always take the first of the bunch.

In addition, some owners within the county are confidential and do not appear in this table. We cannot estimate taxes for those properties because the county associates mill levies with owners, not values.

In [ ]:
# Import values and drop duplicates
col_names = {"strap": "STRAP", "mill_levy": "MILLLEVY",
             "account_type": "ACCTTYPE"}
col_types = {"strap": str, "mill_levy": np.float32, "account_type": str}
own_df = process_csv(OWNERS_URL, col_names, col_types)

### Join Tables
In their [data dictionary](https://assets.bouldercounty.org/wp-content/uploads/2017/02/ar-property-data-download-help.pdf), the county suggests performing `OUTER JOINS` on either the Accounts and Parcels or Values tables. For this analysis, all tax information will be joined to the Accounts and Parcel Numbers dataframe because that will ultimately be joined to the spatial table for parcels. Everything will be joined on the `STRAP`.

In [ ]:
dfs_join_order = [acct_df, val_df, own_df]
tax_df = reduce(lambda l, r: pd.merge(l, r, on=["STRAP"], how="left"), dfs_join_order)
tax_df.tail(20)